In [3]:
import json

with open('/home/nermakovaa/semester_4/SNA/SNA/data/data.json', 'r') as file:
    data = json.load(file)
    
data

[{'vk': [{'groupId': '19817989',
    'posts': [{'id': 2221808,
      'text': 'А еще мне удалось попробовать еще одну #ВВновинку -Пирожки постные с рисом и грибами. Пирожки-великолепные. Хорошее тесто и много начинки! Грибочки в начинке -очень понравились!',
      'views': None,
      'forwards': 0,
      'replies': [{'text': 'Мне они тоже понравились',
        'sender_id': 103700096,
        'date': 1713129164,
        'reactions': [{'emoji': 'like', 'count': 4}],
        'replies': 2,
        'sender_name': 'Любовь',
        'last_name': 'Креминская',
        'sex': 1,
        'city': 'Санкт-Петербург',
        'bdate': '23.4'}],
      'date': 1713127426,
      'reactions': [{'emoji': 'like', 'count': 30}]},
     {'id': 2221793,
      'text': 'Хотела поделиться впечатлением об этом печенье-#ВВновинку Печенье сахарное с изюмом. Очень вкусное печенье,нежное на вкус. Изюм придает пикантность. Пачку очень быстро съели! Всем советуем!',
      'views': None,
      'forwards': 0,
      'repl

## Tg

### Love Rate

In [5]:
def love_rate(data):
    """
    Коэффициент привлекательности (Love Rate)
    (Количество лайков / Общее количество уникальных просмотров ) * 100%
    """
    total_views = 0
    total_likes = 0

    for i in data:
        for item in i['tg']:
            total_views += sum([0 if _["views"] is None else _["views"] for _ in item["posts"]])
            total_likes += sum([reaction["count"] for post in item["posts"] for reaction in post["reactions"] if reaction["emoji"] == "❤"])

    if total_views > 0 and total_likes > 0:
        love_rate = (total_likes / total_views) * 100
        return love_rate
    else:
        return 0
    
love_rate(data)

1.1029618652810864

### Engagement Rate 

In [6]:
def engagement_rate(self):
    '''
    Коэффициент вовлеченности (Engagement Rate)
    (Общее число взаимодействий / Общее число подписчиков) * 100%
    '''
    total_interactions = 0
    total_comments = 0

    for i in data:
        for item in i['tg']:
            followers = item['membersCount']
            likes = sum(reaction['count'] for post in item['posts'] for reaction in post['reactions'])
            reposts = sum(post['forwards'] for post in item['posts'])

            unique_sender_ids = set(comment['sender_id'] for post in item['posts'] for comment in post.get('replies', []))

            total_comments += len(unique_sender_ids)
            total_interactions += likes + total_comments + reposts

    if total_interactions > 0 and followers > 0:
        engagement_rate = (total_interactions / followers) * 100
        return engagement_rate
    else:
        return 0

engagement_rate(data)

0.9626264357963312

### Sentimental Analysis for messages

In [8]:
from transformers import pipeline

def top_regions(data):
    '''
    Топ-20 регионов с наибольшим числом пользователей
    [регион | пользователей | % | тональность]
    '''
    # https://huggingface.co/blanchefort/rubert-base-cased-sentiment модель для анализа тональности
    classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment") 

    city_users = {}

    total_users = 0
    
    for i in data:
        for item in i['tg']:
            for post in item['posts']:
                for reply in post.get('replies', []):
                    city = reply.get('city', 'Unknown')
                    if city not in city_users:
                        city_users[city] = []
                    city_users[city].append(reply['sender_id'])
                    total_users += 1

    city_stats = {}
    for city, users in city_users.items():
        users_count = len(users)
        percent = (users_count / total_users) * 100

        user_messages = [post['text'] for post in item['posts'] for post in post['replies'] if post.get('city') == city and post['text'] is not None]
        classified_messages = classifier(user_messages)
        len_messages = len(classified_messages)
        positive = len([i for i in classified_messages if i['label']=='POSITIVE']) / len_messages
        negative = len([i for i in classified_messages if i['label']=='NEGATIVE']) / len_messages
        neutral = len([i for i in classified_messages if i['label']=='NEUTRAL']) / len_messages

        city_stats[city] = {
            'users_count': users_count,
            'percent': percent,
            'user_messages': user_messages,
            'positive': positive,
            'negative': negative,
            'neutral': neutral
        }

    # Оставляем топ-20, сортируем city_stats по users_count 
    sorted_city_stats = dict(sorted(city_stats.items(), key=lambda x: x[1]['users_count'], reverse=True)[:20])

    top_regions = json.dumps(sorted_city_stats, indent=4, ensure_ascii=False)
    return top_regions

result = top_regions(data)
print(result)

{
    "null": {
        "users_count": 9,
        "percent": 100.0,
        "user_messages": [
            "А почему у вас нет воды легенда гор в 19 литровых бутылках?))",
            "Бенто со сгущенкой",
            "О, словечки вернулись",
            "Добрый день, Алина",
            "Полностью поддерживаем вас в этом выборе",
            "Я на хлебной планете уже 7 лет)",
            "Любимое мороженое. И на Пломбирной планете можно было бы пополнять бесконечно ведерко",
            "Промокодов для постоянных клиентов давно не было("
        ],
        "positive": 0.125,
        "negative": 0.125,
        "neutral": 0.75
    }
}


## ВКонтакте

In [14]:
import json

with open('/home/nermakovaa/semester_4/SNA/SNA/data/data.json', 'r') as file:
    data = json.load(file)
    
data

[{'vk': [{'groupId': '19817989',
    'posts': [{'id': 2221808,
      'text': 'А еще мне удалось попробовать еще одну #ВВновинку -Пирожки постные с рисом и грибами. Пирожки-великолепные. Хорошее тесто и много начинки! Грибочки в начинке -очень понравились!',
      'views': None,
      'forwards': 0,
      'replies': [{'text': 'Мне они тоже понравились',
        'sender_id': 103700096,
        'date': 1713129164,
        'reactions': [{'emoji': 'like', 'count': 4}],
        'replies': 2,
        'sender_name': 'Любовь',
        'last_name': 'Креминская',
        'sex': 1,
        'city': 'Санкт-Петербург',
        'bdate': '23.4'}],
      'date': 1713127426,
      'reactions': [{'emoji': 'like', 'count': 30}]},
     {'id': 2221793,
      'text': 'Хотела поделиться впечатлением об этом печенье-#ВВновинку Печенье сахарное с изюмом. Очень вкусное печенье,нежное на вкус. Изюм придает пикантность. Пачку очень быстро съели! Всем советуем!',
      'views': None,
      'forwards': 0,
      'repl

### Love Rate

In [19]:
def love_rate(data):
    """
    Коэффициент привлекательности (Love Rate)
    (Количество лайков / Общее количество уникальных просмотров) * 100%
    """
    total_views = 0
    total_likes = 0

    for i in data:
        for item in i['vk']:
            valid_posts = [post for post in item["posts"] if post["views"] is not None] # только те посты, где просмотры != None
            total_views += sum([post["views"] for post in valid_posts])
            total_likes += sum([reaction["count"] for post in valid_posts for reaction in post["reactions"] if reaction["emoji"] == "like"])

    if total_views > 0 and total_likes > 0:
        love_rate = (total_likes / total_views) * 100
        return love_rate
    else:
        return 0
    
love_rate(data)

35.0

### Engagement Rate 

In [27]:
def engagement_rate(self):
    '''
    Коэффициент вовлеченности (Engagement Rate)
    (Общее число взаимодействий / Общее число подписчиков) * 100%
    '''
    total_interactions = 0
    total_comments = 0

    for i in data:
        for item in i['vk']:
            followers = item['membersCount']
            likes = sum(reaction['count'] for post in item['posts'] for reaction in post['reactions'])
            reposts = sum(post['forwards'] for post in item['posts'])

            unique_sender_ids = set(comment['sender_id'] for post in item['posts'] for comment in post.get('replies', []))

            total_comments += len(unique_sender_ids)
            total_interactions += likes + total_comments + reposts

    if total_interactions > 0 and followers > 0:
        engagement_rate = (total_interactions / followers) * 100
        return engagement_rate
    else:
        return 0

engagement_rate(data)

0.135772935928105

### Sentimental Analysis for messages

In [28]:
from transformers import pipeline

def top_regions(data):
    '''
    Топ-20 регионов с наибольшим числом пользователей
    [регион | пользователей | % | тональность]
    '''
    # https://huggingface.co/blanchefort/rubert-base-cased-sentiment модель для анализа тональности
    classifier = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment") 

    city_users = {}

    total_users = 0
    
    for i in data:
        for item in i['vk']:
            for post in item['posts']:
                for reply in post.get('replies', []):
                    city = reply.get('city', 'Unknown')
                    if city not in city_users:
                        city_users[city] = []
                    city_users[city].append(reply['sender_id'])
                    total_users += 1

    city_stats = {}
    for city, users in city_users.items():
        users_count = len(users)
        percent = (users_count / total_users) * 100

        user_messages = [post['text'] for post in item['posts'] for post in post['replies'] if post.get('city') == city and post['text'] is not None]
        classified_messages = classifier(user_messages)
        len_messages = len(classified_messages)
        positive = len([i for i in classified_messages if i['label']=='POSITIVE']) / len_messages
        negative = len([i for i in classified_messages if i['label']=='NEGATIVE']) / len_messages
        neutral = len([i for i in classified_messages if i['label']=='NEUTRAL']) / len_messages

        city_stats[city] = {
            'users_count': users_count,
            'percent': percent,
            'user_messages': user_messages,
            'positive': positive,
            'negative': negative,
            'neutral': neutral
        }

    # Оставляем топ-20, сортируем city_stats по users_count 
    sorted_city_stats = dict(sorted(city_stats.items(), key=lambda x: x[1]['users_count'], reverse=True)[:20])

    top_regions = json.dumps(sorted_city_stats, indent=4, ensure_ascii=False)
    return top_regions

result = top_regions(data)
print(result)

{
    "Москва": {
        "users_count": 4,
        "percent": 50.0,
        "user_messages": [
            "Мы только первую быстро скушали, а после второй решили, что наелись и больше не хотим 😂",
            "Надо брать однозначно",
            "Пойду запускать ракету всем хорошего вечера и до новых встреч",
            "Сказочное фото, спокойной ночи"
        ],
        "positive": 0.5,
        "negative": 0.25,
        "neutral": 0.25
    },
    "Санкт-Петербург": {
        "users_count": 1,
        "percent": 12.5,
        "user_messages": [
            "Мне они тоже понравились"
        ],
        "positive": 1.0,
        "negative": 0.0,
        "neutral": 0.0
    },
    "Балашиха": {
        "users_count": 1,
        "percent": 12.5,
        "user_messages": [
            "У нас тоже быстро закончилось."
        ],
        "positive": 0.0,
        "negative": 1.0,
        "neutral": 0.0
    },
    "null": {
        "users_count": 1,
        "percent": 12.5,
        "user_messa